In [8]:
import pandas as pd
df = pd.read_csv('Tweets.csv')
df

,Unnamed: 0,Tweet,Location,year,month,day,Sentiment
0,0,اللي بينا مايتقدر عمرودياب فودافون,"Cairo, Egypt",2022,4,4,negative
1,1,ما بعد فوق قمة الاستهتار والاهمال واللا مبالاة...,Egypt,2022,4,4,negative
2,2,عاجل فضيحة فودافون تتسبب في أزمة بعد تسريببيان...,dokki,2022,4,4,negative
3,3,وحدة فودافون,مصر,2022,4,3,neutral
4,4,فودافون وحده وحده وحده وحده ...,مصر,2022,4,3,positive
...,...,...,...,...,...,...,...
4786,4786,كسم على كسم فودافون كفرتوا الواحد,"Cairo, Egypt",2022,3,27,negative
4787,4787,شنط رمضان رمضان بلا محتاج للتبرع فودافون ك...,tanta,2022,3,27,neutral
4788,4788,الخط فودافون و الراوتر فودافون و لما شبكة زفت ...,cairo,2022,3,27,negative
4789,4789,شركه فودافون شركه حراميه,Banha,2022,3,27,negative


In [9]:
from camel_tools.utils.charsets import AR_LETTERS_CHARSET
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.sentiment import SentimentAnalyzer
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tagger.default import DefaultTagger
from functools import lru_cache


    
class Morphological_Analysis():

    def __init__(self, DF, COL):
        self.df = DF.copy()
        self.col = COL
        self.mled = MLEDisambiguator.pretrained()
        self.tagger = DefaultTagger(self.mled, 'pos')
        self.mapping = {}
        

    def del_nonAR(self):
        for rows in self.df[self.col]:
            for char in rows:
                if char not in AR_LETTERS_CHARSET and char != ' ' :
                    self.df[self.col] = self.df[self.col].str.replace(char, '')
        return self.df

    '''                
    def get_lex(self):
        
        db = MorphologyDB.builtin_db('calima-egy-r13')
        analyzer = Analyzer(db)
        
        for rows in df[self.col]:
            sentence = simple_word_tokenize(rows)
            lex_sentence = ''
            for words in sentence:
                try:
                    lex_sentence += analyzer.analyze(words)[0]['root'] + ' '
                    lex_sentence = lex_sentence.replace('.', '')
                except:
                    lex_sentence += ' ' + words + ' '
            print(lex_sentence)
            self.df[self.col] = self.df[self.col].str.replace(rows, lex_sentence)
        return df
    '''
    
    def replace_rows(self, rows):
        Cleaned_Sent = ' '
        Split_Sent = rows.split()

        for words in Split_Sent:

            if words in self.mapping:
                Tags = self.mapping[words]
            else:
                Tags = self.tagger.tag(words.split())
                self.mapping[words] = Tags

            if Tags[0] != 'conj' and Tags[0] != 'prep':
                Cleaned_Sent = Cleaned_Sent + words + " "
        return Cleaned_Sent
    
    def del_stopwords(self):
        self.df[self.col] = self.df[self.col].apply(self.replace_rows)
        return self.df
            
class Sentiment_Analysis:
    
    def __init__(self, DF, COL):
        self.df = DF
        self.col = COL
    
    def get_sentiment(self):
        sa = SentimentAnalyzer.pretrained()
        sentences = []
        for rows in self.df[self.col]:
            sentences.append(rows)
        sentiments = sa.predict(sentences)
        self.df['Sentiment'] = sentiments
        return self.df

Removing Non-Arabic Characters

In [10]:
Morph = Morphological_Analysis(df, 'Tweet')


In [11]:
Morph.del_stopwords()

,Unnamed: 0,Tweet,Location,year,month,day,Sentiment
0,0,اللي بينا مايتقدر عمرودياب فودافون,"Cairo, Egypt",2022,4,4,negative
1,1,ما بعد فوق قمة الاستهتار والاهمال واللا مبالا...,Egypt,2022,4,4,negative
2,2,عاجل فضيحة فودافون تتسبب أزمة بعد تسريببيانات...,dokki,2022,4,4,negative
3,3,وحدة فودافون,مصر,2022,4,3,neutral
4,4,فودافون وحده وحده وحده وحده وحده,مصر,2022,4,3,positive
...,...,...,...,...,...,...,...
4786,4786,كسم كسم فودافون كفرتوا الواحد,"Cairo, Egypt",2022,3,27,negative
4787,4787,شنط رمضان رمضان محتاج للتبرع فودافون كاش,tanta,2022,3,27,neutral
4788,4788,الخط فودافون الراوتر فودافون لما شبكة زفت فود...,cairo,2022,3,27,negative
4789,4789,شركه فودافون شركه حراميه,Banha,2022,3,27,negative


In [12]:
df

,Unnamed: 0,Tweet,Location,year,month,day,Sentiment
0,0,اللي بينا مايتقدر عمرودياب فودافون,"Cairo, Egypt",2022,4,4,negative
1,1,ما بعد فوق قمة الاستهتار والاهمال واللا مبالاة...,Egypt,2022,4,4,negative
2,2,عاجل فضيحة فودافون تتسبب في أزمة بعد تسريببيان...,dokki,2022,4,4,negative
3,3,وحدة فودافون,مصر,2022,4,3,neutral
4,4,فودافون وحده وحده وحده وحده ...,مصر,2022,4,3,positive
...,...,...,...,...,...,...,...
4786,4786,كسم على كسم فودافون كفرتوا الواحد,"Cairo, Egypt",2022,3,27,negative
4787,4787,شنط رمضان رمضان بلا محتاج للتبرع فودافون ك...,tanta,2022,3,27,neutral
4788,4788,الخط فودافون و الراوتر فودافون و لما شبكة زفت ...,cairo,2022,3,27,negative
4789,4789,شركه فودافون شركه حراميه,Banha,2022,3,27,negative


In [ ]:
Morph.del_nonAR()

Get Words' Root

In [ ]:
#Morph.get_lex(df, 'Tweet')
df.fillna(value=" ", inplace=True)
#df

Get Sentiments

In [ ]:
Sent = Sentiment_Analysis
Sent.get_sentiment(df, 'Tweet')

In [ ]:
df.to_csv('Tweets.csv', index=False)

In [ ]:
df = pd.read_csv('Tweets.csv')
df

In [ ]:
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tagger.default import DefaultTagger

mled = MLEDisambiguator.pretrained()
tagger = DefaultTagger(mled, 'pos')

#tagger.tag('و عملية برقم تم استرداد المبلغ الي محفظة فودا'.split())

df = pd.read_csv('Tweets.csv')

mapping = {}

def Replace_Row(rows):
    Cleaned_Sent = ' '
    Split_Sent = rows.split()

    for words in Split_Sent:

        if words in mapping:
            Tags = mapping[words]
        else:
            Tags = tagger.tag(words.split())
            mapping[words] = Tags

        if Tags[0] != 'conj' and Tags[0] != 'prep':
            Cleaned_Sent = Cleaned_Sent + words + " "
    return Cleaned_Sent

df['Tweet'] = df['Tweet'].apply(Replace_Row)




    


In [ ]:
Morph